# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233811352527                   -0.50    7.0         
  2   -7.249739502118       -1.80       -1.38    1.0   7.68ms
  3   -7.250895823068       -2.94       -1.85    2.0   9.42ms
  4   -7.250968569772       -4.14       -1.87    2.0   9.79ms
  5   -7.251324716062       -3.45       -2.57    1.0   8.20ms
  6   -7.251337358630       -4.90       -3.04    1.0   8.19ms
  7   -7.251338654933       -5.89       -3.65    2.0   9.21ms
  8   -7.251338785701       -6.88       -4.04    3.0   11.1ms
  9   -7.251338797902       -7.91       -4.69    2.0   9.81ms
 10   -7.251338798652       -9.12       -5.18    2.0   10.0ms
 11   -7.251338798696      -10.35       -5.63    1.0   8.34ms
 12   -7.251338798703      -11.14       -6.00    2.0   9.74ms
 13   -7.251338798704      -12.00       -6.51    1.0   8.50ms
 14   -7.251338798705      -12.98       -7.13    2.0   10.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04518543322290685
[ Info: Arnoldi iteration step 2: normres = 0.7768804675259975
[ Info: Arnoldi iteration step 3: normres = 0.6767508353649142
[ Info: Arnoldi iteration step 4: normres = 0.3930727561780659
[ Info: Arnoldi iteration step 5: normres = 0.19983220094599785
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.36e-02, 4.95e-02, 1.19e-01, 1.45e-01, 4.45e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5420176198204847
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.34e-03, 5.86e-02, 5.21e-01, 8.73e-02, 3.48e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10442326842494482
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.29e-04, 1.52e-02, 5.25e-02, 2.85e-02, 7.96e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09314796230463236
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.33e-